In [1]:
# 训练数据的联合概率已知P(X, Y), 求解P(Y|X)后验概率 = 先验p(Y) * 似然P(X|Y) / P(X)
# 模型假设：条件独立，各个维度的X是独立的
# 模型input：训练数据，可以得到联合概率和先验概率；
# 目标：对于一个输入x，选择P(y=ck) p (x|y) 最大的y的类别

# 先验分布用高斯分布或者多项式分布或者伯努利分布；连续一般高斯，离散选择多项式或者伯努利

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from collections import Counter
import math

In [4]:
def create_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['label'] = iris.target
    df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
    data = np.array(df.iloc[:100, :])
    return data[:,:-1], data[:,-1]

In [5]:
X, y = create_data()

In [7]:
trainx, testx, trainy, testy = train_test_split(X, y, test_size=0.3)

In [28]:
# 这里的高斯分布 用来拟合在y=ck的条件下，特征的分布，由于这里特征为连续值，因此使用高斯分布更加合适
# 先验概率从训练集当中获得，是否合理？
class NavieBayes:
    def __init__(self):
        self.model = None
    
    # 这里X指一个维度的特征
    @staticmethod
    def mean(X):
        return sum(X) / float(len(X))
    
    def stdev(self, X):
        avg = self.mean(X)
        return math.sqrt(sum([pow(x - avg, 2) for x in X]) / float(len(X)))
    
    def gaussian_probability(self, x, mean, stdev):
        exponent = math.exp(-(math.pow(x - mean, 2) /
                              (2 * math.pow(stdev, 2))))
        return (1 / (math.sqrt(2 * math.pi) * stdev)) * exponent
    
    # 计算每个特征维度的均值和方差
    def summarize(self, train_data):
        summaries = [(self.mean(i), self.stdev(i)) for i in zip(*train_data)]
        return summaries

    def fit(self, X, y):
        lables = list(set(y))
        data = {label:[] for label in lables}
        
        for f, label in zip(X, y):
            data[label].append(f)
        
        # 对数据进行划分，对于某个label，计算其下不同维度特征的均值和方差，这样就能计算特征的高斯分布概率
        self.model = {
            label: self.summarize(value) for label, value in data.items()
        }
        print(self.model.items())
        
    def calculate_probabilities(self, input_data):
        probabilities = {}
        # model中保存 对应label各个维度特征的均值和方差
        for label, value in self.model.items():
            probabilities[label] = 1
            for i in range(len(value)):
                mean, std = value[i]
                probabilities[label] *= self.gaussian_probability(input_data[i], mean, std)
        return probabilities
    
    def predict(self, X_test):
        # 根据概率排序，选择最大概率返回0位置(label)
        label = sorted(
            self.calculate_probabilities(X_test).items(),
            key=lambda x: x[-1])[-1][0]
        return label
        
    
    def score(self, X_test, y_test):
        right = 0
        for X, y in zip(X_test, y_test):
            label = self.predict(X)
            if label == y:
                right += 1

        return right / float(len(X_test))

In [29]:
model = NavieBayes()
model.fit(trainx, trainy)
print(model.predict([4.4,  3.2,  1.3,  0.2]))

dict_items([(0.0, [(5.0166666666666675, 0.3337497399083464), (3.4305555555555554, 0.33234808095365953), (1.4611111111111112, 0.18897057057438832), (0.25000000000000006, 0.10408329997330662)]), (1.0, [(5.844117647058823, 0.5134606786425915), (2.6911764705882355, 0.31749724135793084), (4.135294117647059, 0.47333755704209063), (1.2823529411764707, 0.1977380742820189)])])
0.0


In [31]:
model.score(testx, testy)

1.0